## NC DEQ Drinking Water Watch Scraper
_by Byron Rice, 12/6/2019_

retrieves DWW non-coliform samples page

In [1]:
import pandas as pd
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import lxml.html as LH
from pathlib import Path

In [2]:
def df_row_to_dict(row, metadata):

    tidy_row = {'Water System No.':metadata.iloc[0,1],
                     'Water System Name':metadata.iloc[1,1],
                     'Lab Sample No.':metadata.iloc[2,1],
                     'Federal Type':metadata.iloc[0,3],
                     'Principal County Served':metadata.iloc[1,3],
                     'Collection Date':metadata.iloc[2,3],
                     'Analyte Code':row[1],
                     'Analyte Name':row[2],
                     'Method Code':row[3],
                     'Less than Indicator':row[4],
                     'Level Type':row[5],
                     'Reporting Level':row[6],
                     'Concentration level':row[7],
                     'Monitoring Period Begin Date':row[8],
                     'Monitoring Period End Date':row[9]}
    return tidy_row
    

In [3]:
def put_in_single_rows(df_list):
    data = df_list[1]
    data.rename(columns=data.iloc[0], inplace = True)
    data.drop(index=0, inplace = True)

    metadata = df_list[0]
    export = []

    for row in data.itertuples():
        dict = df_row_to_dict(row, metadata)
        export.append(dict)

    return export
    
    

In [4]:
def return_tables(soup):
    export_tables = []
    tables = soup.findAll("table")
    for table in tables:
        records = []
        for tr in table.findAll("tr"):
            trs = tr.findAll("td")
            record = []
            for i in range(len(trs)):
                record.append(trs[i].text)
            records.append(record)
        df = pd.DataFrame(data=records)
        export_tables.append(df)
        
    return export_tables[2:4]

In [5]:
def open_links(page):
    session = HTMLSession()
    r = session.get(page)
    
    links = pd.DataFrame(r.html.absolute_links, columns = {'links'}) # returns list of links on page
    links.set_index(links['links'], inplace = True)
    links.drop(['https://www.pwss.enr.state.nc.us/NCDWW2/index.jsp',
                'https://www.pwss.enr.state.nc.us/NCDWW2/Maps/Map_Template.jsp',
                'http://www.ncwater.org/?page=9'], 
               inplace = True, axis = 0)
    
    html = links['links'].apply(requests.get) # opens links
    htmldf = pd.DataFrame(html)
    htmldf.rename(columns = {'links':'response'}, inplace = True)

    export_df = pd.DataFrame()
    
    for row in htmldf.itertuples():
        soup = BeautifulSoup(row[1].text, 'html.parser') # parses html of webpage
        df_list = return_tables(soup)
        if len(df_list) == 2:
            list_rows = put_in_single_rows(df_list)
            for item in list_rows:
                export_df = export_df.append(item, ignore_index = True)
        else:
            export_df = pd.DataFrame()
            print("no table exists at " + str(row))
    return export_df
    

In [6]:
counties = pd.read_csv('counties_list.csv')
counties_list = []
for county in counties['0']:
    x = str(county)
    counties_list.append(x)

In [7]:
def run_script():
    for i, county in enumerate(counties_list):
        for year in range(2010, 2016):
            fn = f'{county}-{year}.pkl'
            print(f'Running county {i+1} of {len(counties_list)}: {county}')
            if not Path(fn).exists():
                url = f'https://www.pwss.enr.state.nc.us/NCDWW2/JSP/SearchDispatch?number=&name=&companyname=&WaterSystemStatusCode=A&county={county}&WaterSystemType=All&SourceWaterType=All&PointOfContactType=None&SampleType=NonTCRAll&stateclassificationcode=All&begin_date=1%2F1%2F{year}&end_date=1%2F2%2F{year+1}&action1=Search+For+Samples'
                page_df = open_links(url)
                page_df.to_pickle(fn)

In [9]:
import logging

def main():
    # get the logger based on the filename; this tells you where the log is coming from
    logger = logging.getLogger(__name__)
    logger.warning("Starting to do stuff...")
    run_script()
    logger.info("Stuff done!")


if __name__ == "__main__":
    # you setup configuration at the main entrypoint to the code
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s: %(levelname)s: %(name)s: %(message)s",
        handlers=[logging.FileHandler("my-log.log"), logging.StreamHandler()],
    )
    main()



2020-02-19 15:20:46,674: WARNING: __main__: Starting to do stuff...


Running county 1 of 100: ALAMANCE
Running county 1 of 100: ALAMANCE


KeyboardInterrupt: 